## 라이브러리

### 뷰티풀숲 import

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import pandas as pd
import numpy as np

## 셀레니움 import

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
from urllib.request import urlopen

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

     |████████████████████████████████| 904 kB 7.6 MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelea

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


### kb 차차차 크롤링

In [ ]:

# 판다스 데이터 프레임 생성 및 컬럼명 지정
result = pd.DataFrame(columns=['연식', '주행거리','연료', '변속기', '연비', '차종', '배기량', '색상', '모델명', '가격', '연도'])

# 경차_기아
# 크롤링할 페이지
for i in range(231, 258):
  url = f'https://www.kbchachacha.com/public/search/main.kbc#!?countryOrder=1&page={i}&sort=-orderDate&useCode=002001&makerCode=102'
  driver.get(url)

  time.sleep(2)

  soup = BeautifulSoup(driver.page_source, 'html.parser')
 
  inner_list = []
  car_name = []
  car_price = []
  car_date = []
# 크롤링할 경로 지정
  for x in range(1, 21) :
    href_selector = f'div.__used-car-list > div.cs-list02.cs-list02--ratio.small-tp.generalRegist > div.list-in > div:nth-of-type({x}) > div.con > div > a.link'  
    href = soup.select_one(href_selector)
    if not href :
        break

    inner_url = 'https://www.kbchachacha.com' + href['href']
    inner_list.append(inner_url)


    name_selector = f'div.__used-car-list > div.cs-list02.cs-list02--ratio.small-tp.generalRegist > div.list-in > div:nth-of-type({x}) > div.con > div > a > strong.tit'
    name = soup.select_one(name_selector)
    target_text_without_child_tags = [bs_object for bs_object in name if isinstance(bs_object, NavigableString)]
    text = "".join(target_text_without_child_tags).strip()
    car_name.append(text)
    if not name :
        break

    
    pay_selector = f'div.__used-car-list > div.cs-list02.cs-list02--ratio.small-tp.generalRegist > div.list-in > div:nth-of-type({x}) > div.con > div > a.link > strong.pay'
    pay = soup.select_one(pay_selector)
    if not pay :
        break

    target_text_without_child_tags = [bs_object for bs_object in pay if isinstance(bs_object, NavigableString)]
    price = "".join(target_text_without_child_tags).strip()
    car_price.append(price)


    date_selector = f'div.__used-car-list > div.cs-list02.cs-list02--ratio.small-tp.generalRegist > div.list-in > div:nth-of-type({x}) > div.con > div > a.link > div.data-line.tp03 > div.first'
    date = soup.select_one(date_selector)
    if not date :
        break

    target_text_without_child_tags = [bs_object for bs_object in date if isinstance(bs_object, NavigableString)]
    date_ym = "".join(target_text_without_child_tags).strip()
    car_date.append(date_ym)



  for i, a in enumerate(inner_list):
    # print(a)
    driver.get(a)

    time.sleep(3)

    soup = bs(driver.page_source, 'html.parser')
    selector = 'div.detail-info-list > div.detail-info01'
    inner = soup.select('table')
    table_html = str(inner)
    table_df_list = pd.read_html(table_html)
    table_df = table_df_list[0]
    te = table_df.values
    t1 = te.reshape(14, 2)
    te1 = np.delete(t1, [0, 9, 10, 11, 12, 13], 0)
    te1 = np.append(te1, np.array([['모델명', car_name[i]]]), axis=0)
    te1 = np.append(te1, np.array([['가격', car_price[i]]]), axis=0)
    te1 = np.append(te1, np.array([['연도', car_date[i]]]), axis=0)
    
    row = [i[1] for i in te1] 
    result = result.append(pd.Series(row, index=result.columns), ignore_index=True)

In [ ]:
result = result.drop('연식', axis=1)

In [ ]:
result = result[['모델명', '차종', '가격', '연도', '주행거리', '연료', '변속기', '연비', '배기량', '색상']]

### 크롤링한 데이터를 csv 저장

In [ ]:
result.to_csv('경차_기아14.csv', index=None, encoding='utf-8-sig')